In [1]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)
os.environ.get('OPENAI_API_KEY')

'sk-08GBKw9V6YPjQbuOojfI1hSWNSXO5LNKqczpX0EtbLTKnFZq'

In [2]:
from langchain_openai.chat_models import ChatOpenAI

In [3]:
chat = ChatOpenAI(
    openai_api_base=os.environ["CHATGPT_API_ENDPOINT"],
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

4.2 Chroma向量數據庫相似度搜索

In [4]:
from langchain.document_loaders import PyPDFLoader

In [5]:
#讀取PDF文件資料
loaders = [
    PyPDFLoader("./data/01.pdf"),
    PyPDFLoader("./data/02.pdf"),
    PyPDFLoader("./data/03.pdf"),
    PyPDFLoader("./data/04.pdf")
]

In [6]:
#將資料個別放入docs，以document格式儲存
docs = []

for loader in loaders:
    docs.extend(loader.load())

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
#分割資料為多個chunk，每個chunk大小為1000
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

splits = text_splitter.split_documents(docs)

len(splits)

109